In [2]:
# Import the Modules into the Notebook
from uncertainties import ufloat
from uncertainties import wrap
from uncertainties.umath import * 
import numpy as np

In [3]:
# Setup the Structure

# Geometry
geometryDict = {}
geometryDict['beamHeight'] = ufloat(100, 0.25)
geometryDict['beamLength'] = ufloat(500, 0.2)
geometryDict['beamThick'] = ufloat(4.5, 0.1)
geometryDict['beamWidth'] = ufloat(50, 0.25)
print geometryDict

materialDict = {}
materialDict['elasticModulus'] = ufloat(200, 6.67)
materialDict['density'] = ufloat(8.8E-6, 0)
print materialDict

tubeDict = {}
tubeDict['tubePos'] = ufloat(0, 0)
tubeDict['tubeMass'] = ufloat(90, 0.08333)
tubeDict['gravity'] = ufloat(9.81, 0)
print tubeDict

beamDict = {}
beamDict['beamDeflection'] = ufloat(0, 0)
beamDict['beamFrequency'] = ufloat(0, 0)
beamDict['beamStress'] = ufloat(0, 0)
beamDict['beamMass'] = ufloat(0, 0)
print beamDict

sourceDict = {}
sourceDict['pos'] = ufloat(0, 0)
sourceDict['beamDistToCL'] = ufloat(600, 0.0667)
sourceDict['omega'] = ufloat(12.56, 0.041867)
sourceDict['force'] = ufloat(0, 0)
print sourceDict

detectorDict = {}
detectorDict['pos'] = ufloat(-0.5, 0.0333)
print detectorDict

xRayDict = {}
xRayDict['fsAlign'] = ufloat(0, 0)
print xRayDict

{'beamHeight': 100.0+/-0.25, 'beamWidth': 50.0+/-0.25, 'beamThick': 4.5+/-0.1, 'beamLength': 500.0+/-0.2}
{'elasticModulus': 200.0+/-6.67, 'density': 8.8e-06+/-0}
{'gravity': 9.81+/-0, 'tubeMass': 90.0+/-0.08333, 'tubePos': 0.0+/-0}
{'beamFrequency': 0.0+/-0, 'beamMass': 0.0+/-0, 'beamDeflection': 0.0+/-0, 'beamStress': 0.0+/-0}
{'force': 0.0+/-0, 'beamDistToCL': 600.0+/-0.0667, 'omega': 12.56+/-0.041867, 'pos': 0.0+/-0}
{'pos': -0.5+/-0.0333}
{'fsAlign': 0.0+/-0}


In [4]:
# To permit calling and transferring information across all subsystems
# Package it into an overall dict so that the
# Analytical and Response Surface can be programmed inside the functional blocks
overallSystemDict = {}
overallSystemDict['xRayDict'] = xRayDict
overallSystemDict['detectorDict'] = detectorDict
overallSystemDict['sourceDict'] = sourceDict
overallSystemDict['beamDict'] = beamDict
overallSystemDict['tubeDict'] = tubeDict
overallSystemDict['materialDict'] = materialDict
overallSystemDict['geometryDict'] = geometryDict
print overallSystemDict


{'detectorDict': {'pos': -0.5+/-0.0333}, 'materialDict': {'elasticModulus': 200.0+/-6.67, 'density': 8.8e-06+/-0}, 'sourceDict': {'force': 0.0+/-0, 'beamDistToCL': 600.0+/-0.0667, 'omega': 12.56+/-0.041867, 'pos': 0.0+/-0}, 'geometryDict': {'beamHeight': 100.0+/-0.25, 'beamWidth': 50.0+/-0.25, 'beamThick': 4.5+/-0.1, 'beamLength': 500.0+/-0.2}, 'tubeDict': {'gravity': 9.81+/-0, 'tubeMass': 90.0+/-0.08333, 'tubePos': 0.0+/-0}, 'xRayDict': {'fsAlign': 0.0+/-0}, 'beamDict': {'beamFrequency': 0.0+/-0, 'beamMass': 0.0+/-0, 'beamDeflection': 0.0+/-0, 'beamStress': 0.0+/-0}}


In [5]:
# Define Analytical Equations
def defineAnalyticalEquations_Lower(overallSystemDict):
    # Equation 1
    L = overallSystemDict['geometryDict']['beamLength']
    w = overallSystemDict['geometryDict']['beamWidth']
    t = overallSystemDict['geometryDict']['beamThick']
    h = overallSystemDict['geometryDict']['beamHeight']
    density = overallSystemDict['materialDict']['density']
    # User Enters Formula
    beamMass = density * L * (2 * (w + 2 * t)*t + 2 * h * t)
    
    # Equation 2
    tubeMass = overallSystemDict['tubeDict']['tubeMass']
    beamDistToCL = overallSystemDict['sourceDict']['beamDistToCL']
    gravity = overallSystemDict['tubeDict']['gravity']
    omega =  overallSystemDict['sourceDict']['omega']
    # User Enters Formula
    force = tubeMass * (beamDistToCL * omega * omega + gravity)
    

    # Pushed back into the Dict
    overallSystemDict['beamDict']['beamMass'] = beamMass
    overallSystemDict['sourceDict']['force'] = force
    
    return overallSystemDict

In [6]:
# 'h', 'w', 't', 'L', 'E', 'F', 
# 'h^2', 'h w', 'h t', 'h L', 'h E', 'h F', 
# 'w^2', 'w t', 'w L', 'w E', 'w F', 
# 't^2', 't L', 't E', 't F', 
# 'L^2', 'L E', 'L F', 
# 'E^2', 'E F', 
# 'F^2'
def calculateResponseValue(h, w, t, L, E, F, coef, intercept):
    term1 = h*coef[0] + w*coef[1] + t*coef[2] + L*coef[3] + E*coef[4] + F*coef[5]
    term2 = h * (h*coef[6] + w*coef[7] + t*coef[8] + L*coef[9] + E*coef[10] + F*coef[11])
    term3 = w * (w*coef[12] + t*coef[13] + L*coef[14] + E*coef[15] + F*coef[16])

    term4 = t * (t*coef[17] + L*coef[18] + E*coef[19] + F*coef[20])
    term5 = L * (L*coef[21] + E*coef[22] + F*coef[23])
    term6 = E * (E*coef[24] + F*coef[25])
    term7 = F * F * coef[26]
    return intercept + term1 + term2 + term3 + term4 + term5 + term6 + term7
    

In [7]:
def calculateCodedValues(h, w, t, L, E, F):
    # 6 Variables: h, w, t, L, E, F
    lb = np.array([80, 40, 2, 400, 180E9, 9000])
    ub = np.array([120, 60, 7, 600, 220E9, 11000])
    avg = 0.5 * (lb + ub)

    loc_array = np.array([h, w, t, L, E, F])
    loc_array_coded = (loc_array - avg) / (lb - ub) * 2.0
    print loc_array, loc_array_coded
    return list(loc_array_coded)
    
# Define Response Surface Equations
def defineResponseSurface(overallSystemDict, freq_rs, deflection_rs, stress_rs):
    L = overallSystemDict['geometryDict']['beamLength']
    w = overallSystemDict['geometryDict']['beamWidth']
    t = overallSystemDict['geometryDict']['beamThick']
    h = overallSystemDict['geometryDict']['beamHeight']
    E = overallSystemDict['materialDict']['elasticModulus']
    F = overallSystemDict['sourceDict']['force']
    
    h_c, w_c, t_c, L_c, E_c, F_c = calculateCodedValues(h, w, t, L, E, F)
    
    
    #x_vars = np.array([h, w, t, L, E, F])
    #x_vars_n = np.array([h.n, w.n, t.n, L.n, E.n, F.n])
    #x_vars_2d = x_vars.reshape(-1,1).transpose()
    #print x_vars_2d.shape
    # beamDeflection
    beamDeflection = calculateResponseValue(h, w, t, L, E, F, deflection_rs.steps[1][1].coef_[0,1:], deflection_rs.steps[1][1].intercept_[0])
    #beamDeflection = wrap(deflection_rs.predict)(x_vars_2d)
    #loc_I = (w * h * h * h/12)
    #beamDeflection = (1.0/(E * loc_I)) * F * L * L * L
    
    # beamFrequency
    beamFrequency = calculateResponseValue(h, w, t, L, E, F, freq_rs.steps[1][1].coef_[0,1:], freq_rs.steps[1][1].intercept_[0])
    #beamFrequency = wrap(freq_rs.predict)(x_vars_2d)
    #beamFrequency = E * w * h * h * h/12
    
    # beamStress
    beamStress = calculateResponseValue(h, w, t, L, E, F, stress_rs.steps[1][1].coef_[0,1:], stress_rs.steps[1][1].intercept_[0])
    #beamStress = wrap(stress_rs.predict)(x_vars_2d)
    #beamStress = F/(w * t)
    
    
    beamDict['beamDeflection'] = beamDeflection
    beamDict['beamFrequency'] = beamFrequency
    beamDict['beamStress'] = beamStress
    return overallSystemDict

In [8]:
# Define Analytical Equations
# Has some dependency on the Response Surface Equations.. So split it into two parts (Lower and Upper)
def defineAnalyticalEquations_Upper(overallSystemDict):
    # Equation 3
    tubePos = overallSystemDict['tubeDict']['tubePos']
    beamDeflection = overallSystemDict['beamDict']['beamDeflection']
    # User Enters Formula
    sourcePos = tubePos - beamDeflection
    
    # Equation 4
    detectorPos = overallSystemDict['detectorDict']['pos']
    # User Enters Formula
    fsAlign = detectorPos - sourcePos
    
    # Pushed back into the Dict
    overallSystemDict['sourceDict']['pos'] = sourcePos
    overallSystemDict['fsAlign'] = fsAlign
    return overallSystemDict
    

In [9]:
# Test out a simple calculation
from sklearn.externals import joblib

# Load the response surfaces
# https://stackoverflow.com/questions/34373606/scikit-learn-coefficients-polynomialfeatures
freq_rs = joblib.load('freq_rs.pkl')
print freq_rs.steps[1][1].coef_.shape
print freq_rs.steps[1][1].intercept_

deflection_rs = joblib.load('deflection_rs.pkl')
stress_rs = joblib.load('stress_rs.pkl')

overallSystemDict = defineAnalyticalEquations_Lower(overallSystemDict)
overallSystemDict = defineResponseSurface(overallSystemDict, freq_rs, deflection_rs, stress_rs)
overallSystemDict = defineAnalyticalEquations_Upper(overallSystemDict)
print overallSystemDict

(1, 28)
[27.50748961]
[100.0+/-0.25 50.0+/-0.25 4.5+/-0.1 500.0+/-0.2 200.0+/-6.67
 8519577.3+/-57344.77197592552] [-0.0+/-0.0125 -0.0+/-0.025 -0.0+/-0.04000000000000001 -0.0+/-0.002
 9.99999999+/-3.335e-10 -8509.5773+/-57.34477197592552]
{'detectorDict': {'pos': -0.5+/-0.0333}, 'materialDict': {'elasticModulus': 200.0+/-6.67, 'density': 8.8e-06+/-0}, 'sourceDict': {'force': 8519577.3+/-57344.77197592552, 'beamDistToCL': 600.0+/-0.0667, 'omega': 12.56+/-0.041867, 'pos': -1.9522639216379871+/-0.004601431400811745}, 'geometryDict': {'beamHeight': 100.0+/-0.25, 'beamWidth': 50.0+/-0.25, 'beamThick': 4.5+/-0.1, 'beamLength': 500.0+/-0.2}, 'tubeDict': {'gravity': 9.81+/-0, 'tubeMass': 90.0+/-0.08333, 'tubePos': 0.0+/-0}, 'fsAlign': 1.4522639216379871+/-0.03361641222582173, 'xRayDict': {'fsAlign': 0.0+/-0}, 'beamDict': {'beamFrequency': 24.380968667980476+/-0.04208879086717574, 'beamMass': 6.2964+/-0.14852282230173783, 'beamDeflection': 1.9522639216379871+/-0.004601431400811745, 'beamStress'

In [24]:
import scipy.stats as st

# Define DPMO and perform the optimization on variables
# Calculate the dpmo given the ufloat object and the lsl & usl
def dpo_lsl_usl(in_ufloat, lsl, usl):
    loc_mean = in_ufloat.nominal_value
    loc_std = in_ufloat.std_dev
    ret_dpo = 0.0

    if(~np.isnan(lsl)):
        z_lsl = (lsl - loc_mean) * 1.0/loc_std
        loc_lslcdf = st.norm.cdf(z_lsl)
        ret_dpo += loc_lslcdf
    
    if(~np.isnan(usl)):
        z_usl = (usl - loc_mean) * 1.0/loc_std
        loc_uslcdf = st.norm.cdf(z_usl)
        ret_dpo += 1.0 - loc_uslcdf

    return ret_dpo

In [35]:
import scipy.optimize as optimize

# params: h, w, L, t
def f(params, overallSystemDict=overallSystemDict):
    # print(params)  # <-- you'll see that params is a NumPy array
    h_nom, w_nom, t_nom, L_nom = params # <-- for readability you may wish to assign names to the component variables
    # Set the values into the Overall System Dict
    L_std = (overallSystemDict['geometryDict']['beamLength']).std_dev
    w_std = (overallSystemDict['geometryDict']['beamWidth']).std_dev
    t_std = (overallSystemDict['geometryDict']['beamThick']).std_dev
    h_std = (overallSystemDict['geometryDict']['beamHeight']).std_dev
    
    overallSystemDict['geometryDict']['beamLength'] = ufloat(L_nom, L_std) 
    overallSystemDict['geometryDict']['beamWidth'] = ufloat(w_nom, w_std)
    overallSystemDict['geometryDict']['beamThick'] = ufloat(t_nom, t_std)
    overallSystemDict['geometryDict']['beamHeight'] = ufloat(h_nom, h_std)
    
    
    overallSystemDict = defineAnalyticalEquations_Lower(overallSystemDict)
    overallSystemDict = defineResponseSurface(overallSystemDict, freq_rs, deflection_rs, stress_rs)
    overallSystemDict = defineAnalyticalEquations_Upper(overallSystemDict)

    # Calculate the DPO of each variable
    beamFrequency = overallSystemDict['beamDict']['beamFrequency']
    beamStress = overallSystemDict['beamDict']['beamStress']
    beamMass = overallSystemDict['beamDict']['beamMass']
    fsAlign = overallSystemDict['fsAlign']
    
    dpo_beamFrequency = dpo_lsl_usl(beamFrequency, np.nan, 55.0)
    bounds_beamFrequency = (np.nan, 55)
    dpo_beamStress = dpo_lsl_usl(beamStress, np.nan, 20)
    bounds_beamStress = (np.nan, 20)
    dpo_beamMass = dpo_lsl_usl(beamMass, np.nan, np.nan)
    bounds_beamMass = (np.nan, 8)
    dpo_fsAlign = dpo_lsl_usl(fsAlign, np.nan, 0.015)
    bounds_fsAlign = (0.4, 0.6)
    dpo_avg = 0.25 * (dpo_beamFrequency + dpo_beamStress + dpo_beamMass + dpo_fsAlign)

    
    
    return dpo_avg

h_Nom_init = (overallSystemDict['geometryDict']['beamHeight']).nominal_value
w_Nom_init = (overallSystemDict['geometryDict']['beamWidth']).nominal_value
t_Nom_init = (overallSystemDict['geometryDict']['beamThick']).nominal_value
L_Nom_init = (overallSystemDict['geometryDict']['beamLength']).nominal_value

initial_guess = [h_Nom_init, w_Nom_init, t_Nom_init, L_Nom_init]
result = optimize.minimize(f, initial_guess, method='BFGS', options={'gtol': 1e-6, 'disp': True})
if result.success:
    fitted_params = result.x
    print(fitted_params)
else:
    raise ValueError(result.message)

Optimization terminated successfully.
         Current function value: 0.750000
         Iterations: 0
         Function evaluations: 6
         Gradient evaluations: 1
[100.   50.    4.5 500. ]


In [36]:
a = ufloat(2, 0.1)
print a
a = 3
a.std_dev = 1
print a

2.00+/-0.10


AttributeError: 'int' object has no attribute 'std_dev'